<a href="https://colab.research.google.com/github/eryeme/NLP_TP/blob/main/TP3_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [3]:
!kaggle datasets download -d nltkdata/movie-review

 57% 4.00M/7.07M [00:00<00:00, 6.08MB/s]
100% 7.07M/7.07M [00:01<00:00, 7.07MB/s]


In [ ]:
!unzip /content/movie-review.zip

In [5]:
import pandas as pd
dataset = pd.read_csv('/content/movie_review.csv')
dataset

,fold_id,cv_tag,html_id,sent_id,text,tag
0,0,cv000,29590,0,films adapted from comic books have had plenty...,pos
1,0,cv000,29590,1,"for starters , it was created by alan moore ( ...",pos
2,0,cv000,29590,2,to say moore and campbell thoroughly researche...,pos
3,0,cv000,29590,3,"the book ( or "" graphic novel , "" if you will ...",pos
4,0,cv000,29590,4,"in other words , don't dismiss this film becau...",pos
...,...,...,...,...,...,...
64715,9,cv999,14636,20,that lack of inspiration can be traced back to...,neg
64716,9,cv999,14636,21,like too many of the skits on the current inca...,neg
64717,9,cv999,14636,22,"after watching one of the "" roxbury "" skits on...",neg
64718,9,cv999,14636,23,"bump unsuspecting women , and . . . that's all .",neg


In [6]:
data = dataset[['text','tag']]
data

,text,tag
0,films adapted from comic books have had plenty...,pos
1,"for starters , it was created by alan moore ( ...",pos
2,to say moore and campbell thoroughly researche...,pos
3,"the book ( or "" graphic novel , "" if you will ...",pos
4,"in other words , don't dismiss this film becau...",pos
...,...,...
64715,that lack of inspiration can be traced back to...,neg
64716,like too many of the skits on the current inca...,neg
64717,"after watching one of the "" roxbury "" skits on...",neg
64718,"bump unsuspecting women , and . . . that's all .",neg


*Pre-processing des données textuelles*

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string

In [ ]:
nltk.download('stopwords')

In [9]:
stop_words = set(stopwords.words('english'))

In [10]:
def preprocess_text(text):
    # Convertir le texte en minuscules
    text = text.lower()
    # Supprimer la ponctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Supprimer les stopwords
    words = text.split()
    filtered_words = [word for word in words if word.lower() not in stop_words]
    # Rejoindre les mots filtrés
    return ' '.join(filtered_words)
data['text'] = data['text'].apply(preprocess_text)
data

<ipython-input-10-c7fd6dfe66b8>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = data['text'].apply(preprocess_text)


,text,tag
0,films adapted comic books plenty success wheth...,pos
1,starters created alan moore eddie campbell bro...,pos
2,say moore campbell thoroughly researched subje...,pos
3,book graphic novel 500 pages long includes nea...,pos
4,words dont dismiss film source,pos
...,...,...
64715,lack inspiration traced back insipid characters,neg
64716,like many skits current incarnation saturdayni...,neg
64717,watching one roxbury skits snl come away chara...,neg
64718,bump unsuspecting women thats,neg


*Entraînement du modéle Word2Vec*

In [14]:
import gensim
import numpy as np
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [13]:
# Tokenisation des reviews
tokenized_reviews = data['text'].apply(word_tokenize)

# Entraînement du modèle Word2Vec
model = Word2Vec(tokenized_reviews, vector_size=100, window=5, min_count=1, workers=4)

*Vectorisation des reviews de movies*

In [15]:
#une fonction qui prend un review et le renvoie le vecteur moyen des embeddings de ces mots des le modéle Word2Vec
def get_vector(review):
    #création d'une liste de vecteurs d'embeddings pour chaque mot de la liste review
    vectors = [model.wv[word] for word in review if word in model.wv]
    #Calcul de la moyenne des vecteurs d'embeddings pour former un vecteur représentatif de la phrase
    #si aucun vecteur n'a été trouvé renvoie d'un vecteur nul de la ^m dimension que les embeddings du modéle
    return np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)
review_vectors = tokenized_reviews.apply(get_vector)

In [16]:
review_vectors[0]

array([-0.36371648,  0.6417614 ,  0.38633883,  0.4228333 ,  0.05931502,
       -1.0912379 ,  0.1707284 ,  0.75464654, -0.18636438, -0.4186831 ,
       -0.03220687, -0.8158033 , -0.01872123,  0.19476788,  0.23674382,
       -0.61155486, -0.08907805, -0.67465097,  0.01174713, -1.0803459 ,
        0.20311002,  0.46298653,  0.41562724, -0.37603074, -0.10459023,
       -0.03716743, -0.20111929, -0.23715895, -0.6684641 ,  0.01778458,
        0.31892166,  0.16699412, -0.06135041, -0.3563558 , -0.13618743,
        0.69687426, -0.04503612, -0.29398543, -0.36155894, -0.8670311 ,
        0.19000737, -0.62112933, -0.39583188,  0.11188901,  0.37279356,
       -0.44578096, -0.55135345,  0.02513816,  0.12458064,  0.32748035,
        0.17946798, -0.41516998, -0.09172398, -0.25158286, -0.42894885,
        0.36995202,  0.35681438, -0.01031612, -0.43526855,  0.27626672,
        0.16453835,  0.16304705, -0.04662032,  0.08492946, -0.42137945,
        0.50987345,  0.17710412,  0.2095899 , -0.6290276 ,  0.47

In [19]:
data['text'] = review_vectors

<ipython-input-19-34d7085f5587>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['text'] = review_vectors


*Division des données*

In [17]:
from sklearn import preprocessing

In [20]:
le = preprocessing.LabelEncoder()
data['tag'] = le.fit_transform(data['tag'])
data

<ipython-input-20-9318e05033cf>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['tag'] = le.fit_transform(data['tag'])


,text,tag
0,"[-0.36371648, 0.6417614, 0.38633883, 0.4228333...",1
1,"[-0.13431495, 0.33491918, 0.3672966, 0.3573626...",1
2,"[-0.3336794, 0.6734943, 0.39663976, 0.34276313...",1
3,"[-0.21127349, 0.3675331, 0.29386392, 0.3224443...",1
4,"[-0.48458967, 0.7070361, 0.3410403, 0.4778235,...",1
...,...,...
64715,"[-0.18670432, 0.39399108, 0.44572863, 0.111174...",0
64716,"[-0.2855242, 0.5124587, 0.35325256, 0.2741174,...",0
64717,"[-0.27290803, 0.4903287, 0.44689843, 0.2815295...",0
64718,"[-0.23052546, 0.50810283, 0.31702673, 0.266856...",0


In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(review_vectors, data['tag'], test_size=0.2, random_state=42)

*Construction d'un classificateur*

In [25]:
from sklearn.linear_model import LogisticRegression

In [24]:
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train.tolist(), y_train)
y_pred = clf.predict(X_test.tolist())

*Evaluation du modéle*

In [26]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [27]:
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

Accuracy: 0.5730067985166872
Precision: 0.5657365510306687
Recall: 0.6847710330138446
F1 Score: 0.6195884093881203
